In [1]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding, TimeDistributed, concatenate, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
import word2vec_utils as w2v
import data
import numpy as np
import re
import random
from data_utils import split_dataset 
from chat_constants import *


Using Theano backend.


In [2]:
w2v_model = w2v.initialize()

In [47]:
def null_punisher(y_true, y_pred ):
    '''Provides stronger incentive to avoid Null'''
    L = 3
    error = keras.losses.cosine_proximity( y_true,y_pred)
    if y_true[-1] != BLANK[-1] or y_pred[-1] == BLANK[-1]:
        error *= L
    return error

In [73]:
LSTM_DROPOUT = 0.15

In [74]:
A1_input = Input(shape=(MAX_SENT_LENGTH,EMBED_DIM))

A1_layer1 = LSTM(EMBED_DIM - 26,name = "A1_layer1", return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
A1_layer2 = LSTM(EMBED_DIM - 51,name = "A1_layer2", return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
A1_net = A1_layer2(A1_layer1(A1_input))

In [75]:
B_input = Input(shape=(MAX_SENT_LENGTH,EMBED_DIM))

B_layer1 = LSTM(EMBED_DIM-25,name = "B_layer1", return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
B_layer2 = LSTM(EMBED_DIM-50,name = "B_layer2", return_sequences=True, dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)

B_net = B_layer2(B_layer1(B_input))

In [76]:
combined = concatenate([A1_net,B_net])
A2_pred1 = LSTM(EMBED_DIM, name = "A2_layer1", return_sequences = True, dropout = LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
A2_pred2 = LSTM(EMBED_DIM, name = "A2_layer2", return_sequences = True, dropout = LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
#A2_pred3 = LSTM(EMBED_DIM, name = "A2_layer3", return_sequences = True, dropout = LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)

A2_net =A2_pred2(A2_pred1(combined))

chat_model = Model(inputs = [ A1_input,B_input], outputs = [A2_net])
chat_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 30, 101)       0                                            
____________________________________________________________________________________________________
input_9 (InputLayer)             (None, 30, 101)       0                                            
____________________________________________________________________________________________________
A1_layer1 (LSTM)                 (None, 30, 75)        53100                                        
____________________________________________________________________________________________________
B_layer1 (LSTM)                  (None, 30, 76)        54112                                        
___________________________________________________________________________________________

In [77]:
checkpointer = ModelCheckpoint( 'chat_net.h5',verbose = 1,monitor = 'val_acc',save_best_only = True)
early_stopping = EarlyStopping( monitor = 'val_acc',patience = 2)


In [78]:
A1, B, A2 = data.load_data()
All_data = np.array([A1,B,A2])

In [79]:
[A1_set1,B_set1,A2_set1] = All_data[:,0:8000]
[A1_set2,B_set2,A2_set2] = All_data[:,8000:16000]
[A1_set3,B_set3,A2_set3] = All_data[:,16000:24000]
[A1_set4,B_set4,A2_set4] = All_data[:,24000:]
sets = [[A1_set1,B_set1,A2_set1],[A1_set2,B_set2,A2_set2],[A1_set3,B_set3,A2_set3],[A1_set4,B_set4,A2_set4]]

In [80]:
adam = keras.optimizers.Adam(lr = 0.0015)
chat_model.compile( optimizer=adam,loss = null_punisher,metrics = ['accuracy'])
chat_model.load_weights('chat_net.h5',by_name = True)

In [ ]:
for a1,b,a2 in sets:
    A1_train,B_train,A2_train = w2v.get_training_data(a1,b,a2)
    chat_model.fit([A1_train,B_train], A2_train,
          batch_size=200, epochs=3,
          validation_split = 0.05,
          callbacks = [checkpointer,early_stopping] )
    #chat_model.save('chat_net.h5')

Train on 7600 samples, validate on 400 samples
Epoch 1/3
7000/7600 [==========================>...] - ETA: 7s - loss: -0.0128 - acc: 0.3582 

In [71]:
def just_model_un_vectorize( predicted ):
    ret =[]
    for word_vec in predicted:
        if abs(word_vec[-1] - 1) < 0.001:
            ret.append("_")
            continue
        word_vec = word_vec[:-1]
        x = w2v_model.similar_by_vector( word_vec, topn = 3 )
        print( x )
        w2v_word,w2v_similarity = x[0]
        print( w2v_word, " ", w2v_similarity )
        ret.append(w2v_word)
    return( " ".join(ret) )
         

In [62]:
def get_random_test_sentence():
    index = random.randint(0,len(A1)-1)
    print A2[index]
    return( w2v.get_training_data(A1[index:index+1],B[index:index+1],A2[index:index+1]))

In [63]:
A1_test,B_test,A2_test = get_random_test_sentence()

2 7 2 @ Would you please see if Sandro is with him ? He's not in his room . I'm sorry to disturb you .


In [31]:
w2v.unvectorize_initialize()

In [64]:
predicted = chat_model.predict([A1_test,B_test])

In [72]:
predicted_words = just_model_un_vectorize( predicted[0] )
print( predicted_words )

[(u'1', 0.9793657064437866), (u'2', 0.7656130194664001), (u'4', 0.5564773082733154)]
(u'1', ' ', 0.9793657064437866)
[(u'9', 0.8987573385238647), (u'8', 0.8393089771270752), (u'7', 0.6883097887039185)]
(u'9', ' ', 0.8987573385238647)
[(u'1', 0.9815201163291931), (u'2', 0.7573592662811279), (u'4', 0.5622636079788208)]
(u'1', ' ', 0.9815201163291931)
[(u'@', 0.9998001456260681), (u'1', 0.5307570099830627), (u'Ears', 0.48693588376045227)]
(u'@', ' ', 0.9998001456260681)
[(u'You', 0.7442670464515686), (u'But', 0.7287113666534424), (u'Well', 0.684301495552063)]
(u'You', ' ', 0.7442670464515686)
[(u'.', 0.9013633728027344), (u',', 0.8703750371932983), (u'--', 0.69184809923172)]
(u'.', ' ', 0.9013633728027344)
[(u'.', 0.8808210492134094), (u',', 0.8138697147369385), (u'you', 0.7223097085952759)]
(u'.', ' ', 0.8808210492134094)
[(u'.', 0.9075521230697632), (u',', 0.8332178592681885), (u'that', 0.6965234279632568)]
(u'.', ' ', 0.9075521230697632)
[(u'.', 0.9146542549133301), (u',', 0.8310866355

In [35]:
w2v_model.most_similar('')

[(u',', 0.8113983869552612),
 (u'--', 0.5987824201583862),
 (u'GALLERY', 0.5957099199295044),
 (u'charts', 0.5932552814483643),
 (u'hellhole', 0.5896182656288147),
 (u'Ears', 0.5814112424850464),
 (u'contributions', 0.5766328573226929),
 (u'Underworld', 0.5752488970756531),
 (u'GAP', 0.5732450485229492),
 (u'and', 0.5639781951904297)]

In [16]:
a = {}
a['__BLANK__'] = BLANK
a['__BLANK__']

import pickle
with open('unknown_words_stored.pkl', 'w+') as f:
    pickle.dump(a, f)

In [17]:
print( A1[30000], B[30000],A2[30000] )

('1 9 1 @ Yes I am .', '1 9 1 @ What are you doing ?', '1 9 1 @ Nothing .')


In [32]:
w2v_model.similar_by_word("man")

[(u'woman', 0.7161317467689514),
 (u'guy', 0.6970781087875366),
 (u'warrior', 0.6738820672035217),
 (u'fellow', 0.6728172302246094),
 (u'hero', 0.6715325117111206),
 (u'boy', 0.6687963008880615),
 (u'lady', 0.6600995063781738),
 (u'cat', 0.656214714050293),
 (u'person', 0.6318941116333008),
 (u'kid', 0.6249631643295288)]

In [41]:
len(A1)

106455

In [22]:
A2.shape

(32269,)

In [ ]:
AB_Input = Input(shape =(MAX_SENT_LENGTH,EMBED_DIM))

AB_layer1 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer1", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
AB_layer2 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer2", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
AB_layer3 = LSTM(EMBED_DIM,return_sequences=True, name = "AB_layer3", dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)

AB_output = AB_layer3(AB_layer2(AB_layer1(AB_Input)))

chat_model = Model(inputs = [ AB_Input], outputs = [AB_output])
chat_model.compile(loss=null_punisher,
              optimizer='Adam',
              metrics=['accuracy']
              )
early_stopping = EarlyStopping( monitor = 'val_acc',patience = 2)
chat_model.load_weights('chat_net.h5',by_name=True)
chat_model.fit([A1B_train], A2_train,
          batch_size=200, epochs=20,
          validation_split = 0.025 )
chat_model.save('chat_net.h5')